# Atmospheric Correction in OBIA4RTM using Google Earth Engine and 6S

This notebook shows how to use Google-Earth-Engine (GEE) and the 6S algorithm (REFERENCE!) to correct Sentinel-2 imagery for atmospheric effects and convert to **`surface reflectance values`**. Moreover, a **`cloud mask and a cloud shadow mask`** is computed using code made avialable by Sam Murphy under Apache 2.0 license (https://github.com/samsammurphy).

This functionality is part of the **optional pre-processing facilities** offered by OBIA4RTM. The reason for including such functionalities is based on the fact that a tool that claims to be **operational** clearly requires **integrated image pre-processing chains** that make use of **state-of-the-art methods** in a **highly efficient way**.

Of course, **users of OBIA4RTM are not forced to use this offer** as also user-defined and processed Sentinel-2 files can be provided to the software (however, atmospheric correction then needs to be done by the user as well). By using GEE, however, there are some distinct advantages:

- no download of imagery
- the computationally intensive part of the image correction is done on the GEE cloud

Of course, this also has some trade-offs. GEE is a commercial solution and propietary to Google. Therefore, it is not guaranteed that the API will be available and maintained in the future. Also the business model might change.

**`NOTE: If you don't want to use GEE, then please skip this notebook.`**

Moreover, the installation of the GEE client API and the 6S software might be time-consuming and is not recommended to unexperienced users.

Therefore, there is always this *traditional* workflow with user-provided files possible in OBIA4RTM.

## Checking if everything is working

Try to import the according **`OBIA4RTM`** module to perform the atmospheric correction

In [1]:
from OBIA4RTM.S2_PreProcessor.s2_Py6S_attcor import s2_Py6S_atcorr

Make also sure to have **Google Earth Engine Python API** installed and have the required credentials. In case you get an error you have to follow these instructions: https://developers.google.com/earth-engine/python_install

In [2]:
import ee
ee.Initialize()

Moreover, it is necessary to check if **Py6S** (the Python wrapper around Py6S) is working. Therefore, you can try the following test (taken from https://py6s.readthedocs.io/en/latest/installation.html where you can also find some more hints about the installation of Py6S):

In [3]:
import Py6S
Py6S.SixS.test()

6S wrapper script by Robin Wilson
Using 6S located at /usr/local/bin/sixs
Running 6S using a set of test parameters
6sV version: 1.1
The results are:
Expected result: 619.158000
Actual result: 619.158000
#### Results agree, Py6S is working correctly


0

If that produces an error (something like *6S executable not found*) you can do the following:

- make sure that you run OBIA4RTM.PreProcessor.install_6S and it terminated without error messages

- if that is/ was the case, then the problem is related to the system path. Most likely, OBIA4RTM is not able to find the 6S executable. Per Default OBIA4RTM.PreProcessor.install_6S builds and installs 6S to the OBIA4RTM user directory. You can find the OBIA4RTM user directory (where also all the config files are stored) via:

In [ ]:
import OBIA4RTM
import os
# get the user-directory of OBIA4RTM
with open(os.path.dirname(OBIA4RTM.__file__) + os.sep + 'OBIA4RTM_HOME') as data:
    info = data.readline()
# info contains the desired user-directory
print('OBIA4RTM Directory: {}'.format(info))

In this directory, a sub-folder should contain the 6S code and the executable Py6S is searching for. It should be found this way:

In [ ]:
sixS_dir = info + os.sep + 'sixS' + os.sep + 'src' + os.sep + '6SV1.1'
print('Directory containing the 6S executable: {}'.format(sixS_dir))

Go this directory and look for the executable. The executable will be **`sixsV1.1`** on Unix\OS X system an **`sixsV1.1.exe`** on Windows.

No either copy this executable to a location that is part of your **`System\Path`** **OR** create a **`symbolic link (recommended for Unix/OS X)`** by opening a terminal in the directory of the executable and typing in:

```{sh}
$ sudo ln sixsV1.1 /usr/local/bin/sixs
```


On **Windows** you can also create a symbolic link (never tested from my side) by opening the cmd window and typing in:
```{sh}
$ MKLINK sixsV1.1.exe C:\Windows\System
```

Set up a new atcorr object to be able to run the 6S algorithm on a user-defined geometry and a user-defined date (Google Earth-Engine will then search for the proper Sentinel-2 scene)

## Executing the atmospheric correction using GEE and the 6S algorithm

In [4]:
# determine geometry and date first
# in this example, a polygon covering the northern Alps in Souhtern Germany/ Tyrol will be used
geom = ee.Geometry.Polygon(
  [[11, 48], [12, 48], [12, 48.5], [11, 48.5], [11, 48.5]]
)
date = "2017-07-06"
# create an class instance of the s2_Py6S_atcorr class in OBIA4RTM
at = s2_Py6S_atcorr()
# now run the 6S algorithm for the selected region and time on Sentinel-2 imagery
# you will get back the surface reflectance of the input that was top-of-atmosphere (L1 level)
S2_surf = at.run_py6s(geom, date)

The output of the above code snippet should be a Google Earth Engine image.Image containing Sentinel-2 atmospherically surface reflectance values. To check this run:

In [5]:
# This should print the image's band names
# if everything is OK, it should output B2, B3, B4, B5, B6, B7, B8A, B11, B12, CloudMask, ShadowMask
S2_surf.bandNames().getInfo()

['B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8A',
 'B11',
 'B12',
 'CloudMask',
 'ShadowMask']

## Display the results

**`Reflectance Values`**

In [7]:
from IPython.display import display, Image

region = geom.buffer(5000).bounds().getInfo()['coordinates']
channels = ['B4','B3','B2']

# convert to % reflectance
original = Image(url=at.S2.select(channels).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.5
                }))

corrected = Image(url=S2_surf.select(channels).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.5
                }))

display(original, corrected)

**`Cloud and Shadow Masks`**

In [8]:
# Display the cloud mask and the shadow mask
cm = Image(url=at.S2.select('CloudMask').getThumbUrl({
                'region':region
                }))

sm = Image(url=at.S2.select('ShadowMask').getThumbUrl({
                'region':region
                }))

display(cm, sm)